# DocuSage

# Assignment: Part 1

## Setup and Installation
#### This section installs the necessary libraries for the project, including `langchain`, `sentence_transformers`, and `pinecone`. These libraries are essential for text processing, embedding generation, and vector storage.

In [1]:
!pip install langchain
!pip install langchain_groq
!pip install sentence_transformers
!pip install langchain_community
!pip install pypdf
!pip install xformers
!pip install langchain_huggingface
!pip install pinecone pinecone_client[grpc]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 946.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255

## Importing Libraries
#### Here, we import the required modules for text splitting, vector storage, and retrieval-based QA systems. The imports also include environment variable management and model loading.

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import os
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone as PC
from langchain.prompts import PromptTemplate

## Environment Variables
#### This section loads API keys from environment variables. These keys are crucial for accessing external services like Pinecone and Groq.

In [3]:
from google.colab import userdata
GROQ_KEY = userdata.get('GROQ_API_KEY')
PINECONE_KEY = userdata.get('PINECONE_API_KEY')

## Document Loading and Preprocessing
#### We load a PDF document and split it into chunks for processing. This is a common preprocessing step in NLP tasks to handle large documents.

In [4]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
data = loader.load()

In [5]:
# Display the first document to verify loading
data[0]

Document(metadata={'source': '/content/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks s

## Text Splitting
#### The document is split into smaller chunks using a recursive character splitter. This is essential for efficient processing and embedding generation.

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [7]:
# Verify the text chunks
text_chunks

[Document(metadata={'source': '/content/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or'),
 Document(metadata={'source': '/content/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'page': 0}, page_content='convolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attent

## Embedding Generation
#### We initialize a HuggingFace model to generate embeddings for the text chunks. The embeddings are used for similarity search and retrieval tasks.

In [8]:
model_name = "dunzhang/stella_en_400M_v5"
model_kwargs = {'device': 'cuda', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/169k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/186 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

In [9]:
# Check the dimensionality of the embeddings
len(embedding.embed_query("How are you"))

1024

## Vector Store Initialization
#### We create a Pinecone vector store to index the document embeddings. This allows for efficient similarity search and retrieval.

In [11]:
# Verify the vector store initialization
docsearch

## Retrieval and Query Processing
#### The vector store is used to perform similarity search and retrieve relevant documents based on a query. This is a key component of retrieval-based QA systems.

In [12]:
docsearch.as_retriever()
query = "What are self-attention layers?"
docs = docsearch.similarity_search(query)

In [13]:
# Display retrieved documents
docs

[Document(metadata={}, page_content='[31, 2, 8].\n•The encoder contains self-attention layers. In a self-attention layer all of the keys, values\nand queries come from the same place, in this case, the output of the previous layer in the\nencoder. Each position in the encoder can attend to all positions in the previous layer of the\nencoder.\n•Similarly, self-attention layers in the decoder allow each position in the decoder to attend to\nall positions in the decoder up to and including that position. We need to prevent leftward'),
 Document(metadata={}, page_content='[31, 2, 8].\n•The encoder contains self-attention layers. In a self-attention layer all of the keys, values\nand queries come from the same place, in this case, the output of the previous layer in the\nencoder. Each position in the encoder can attend to all positions in the previous layer of the\nencoder.\n•Similarly, self-attention layers in the decoder allow each position in the decoder to attend to\nall positions in th

## Prompt Template Definition
#### This cell defines a prompt template for the language model to use during the question-answering process. The template includes placeholders for `context` and `question`, which are dynamically filled with relevant information during execution. The model is instructed to answer the question based on the provided context and to respond with "I don't know" if the context lacks sufficient information. This ensures that the model's responses are grounded in the available data, enhancing the reliability and accuracy of the answers.

In [14]:
prompt_template = """
Context: {context}
Question: {question}
Answer the question based on the provided context. If the context doesn't contain enough information, say "I don't know."
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

## Language Model and QA Chain
#### We initialize a language model and create a QA chain to process queries and generate answers. The model uses the retrieved documents to provide context-aware responses.

In [15]:
llm = ChatGroq(temperature=0, groq_api_key=GROQ_KEY, model_name="llama-3.1-70b-versatile")
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs={"prompt": PROMPT})

In [16]:
# Test the QA system with a sample query
qa.invoke(query)

{'query': 'What are self-attention layers?',
 'result': 'According to the provided context, self-attention layers are layers in which all of the keys, values, and queries come from the same place. In the encoder, this means that each position in the encoder can attend to all positions in the previous layer of the encoder. In the decoder, each position can attend to all positions in the decoder up to and including that position.'}

## Interactive QA System
#### An interactive loop is set up to continuously accept user queries and provide answers using the QA system. This simulates a real-time QA bot.

In [18]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    break
  if user_input == '':
    continue
  result = qa.invoke({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: What are the benefits of attention mechanisms?
Answer: The provided context does not explicitly mention the benefits of attention mechanisms. However, it does mention a paper titled "Attention Is All You Need" and another paper that questions whether active memory can replace attention, implying that attention mechanisms are important and potentially powerful. But without more information, I don't know the specific benefits of attention mechanisms.
Input Prompt: How does the Transformer differ from RNNs and CNNs?
Answer: According to the provided context, the Transformer differs from RNNs and CNNs in that it relies entirely on an attention mechanism, dispensing with recurrence (found in RNNs) and convolutions (found in CNNs) entirely.
Input Prompt: How does self-attention work?
Answer: I don't know. The provided context describes what self-attention is and its applications, but it does not explain how self-attention works.
Input Prompt: Describe the model architecture.
An